In [ ]:
import torch

Load uni-modal models

In [ ]:
from ellzaf_ml.models import GhostFaceNetsV2
import wespeaker

In [ ]:
model_v = GhostFaceNetsV2(image_size=112, width=1, dropout=0.)
model_a = wespeaker.load_model("campplus").model

Check expected input shapes

In [ ]:
model_v(torch.randn(2, 3, 112, 112)).shape

In [ ]:
model_a(torch.randn(2, 3, 80)).shape

Load fusion module

In [ ]:
from synthweave.fusion import CFF, AFF, IAFF, CAFF, GFF

# or

from synthweave.utils import get_fusion

In [ ]:
fusion = GFF(
    output_dim=256,
    # dropout=True,
    # input_dims=[512, 192], # infer automatically during 1st forward pass
)

# or

fusion = get_fusion("gff", output_dim=256)

Construct pipeline

In [ ]:
from synthweave.pipeline import BasePipeline

In [ ]:
pipe = BasePipeline(
    models={
        "video": model_v,
        "audio": model_a
    },
    fusion=fusion,
    head=torch.nn.Sequential(torch.nn.Linear(256, 1), torch.nn.Sigmoid())
)
pipe.eval();

Run the pipeline

In [ ]:
dummy_v = torch.randn(2, 3, 112, 112)
dummy_a = torch.randn(2, 3, 80)

inputs = {
    "video": dummy_v,
    "audio": dummy_a
}

pipe(inputs)